In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import accuracy_score
import anndata as ad
import pandas as pd

In [ ]:
accuracy_vector.mean()

0.48635147190008926

In [2]:
input_dir = "/Users/apple/Desktop/KB/data"
adata_train = ad.read_h5ad(input_dir+'/BiddyData/Biddy_train_lineage.h5ad')
adata_test = ad.read_h5ad(input_dir+'/BiddyData/Biddy_test_lineage.h5ad')

# train_labels = adata_train.obs["clone_id"].to_numpy()
test_labels = adata_test.obs["clone_id"].to_numpy()

# train_embeddings = np.load(input_dir+'/feat_RECOMB/train_test_lineage/biddy_train_test_lineage/Biddy_scvi_train_lineage_embeddings.npy')
test_embeddings = np.load(input_dir+'/feat_RECOMB/train_test_lineage/biddy_train_test_lineage/Biddy_scvi_test_lineage_embeddings.npy')

# print(train_labels.shape, train_embeddings.shape)
print(test_labels.shape, test_embeddings.shape)

(613,) (613, 10)


In [7]:
knn_graph = kneighbors_graph(test_embeddings, 5, mode='connectivity', include_self=True)
knn_graph.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [8]:
knn_graph.shape

(613, 613)

In [9]:
accuracy_vector = np.zeros(knn_graph.shape[0])

# Loop over each row (test sample)
for i in range(knn_graph.shape[0]):
    # Convert the sparse row to a dense array
    knn_row_dense = knn_graph[i].toarray().flatten()  # Flatten to ensure it's 1D
    
    # Get the indices of the neighbors for the i-th test sample
    neighbor_indices = np.where(knn_row_dense > 0)[0]
    
    # Get the labels of the neighbors
    neighbor_labels = test_labels[neighbor_indices]
    
    # Compare the neighbor labels to the label of the current test sample
    correct_matches = np.sum(neighbor_labels == test_labels[i])
    
    # Calculate the percentage of correct matches
    accuracy_vector[i] = correct_matches / len(neighbor_indices)




In [10]:
accuracy_vector.mean()

0.45742251223491026